In [1]:
from distributed.diagnostics.plugin import WorkerPlugin
class S3DownloadPlugin(WorkerPlugin):
    def __init__(self, s3Url, filename):
        self.s3Url = s3Url
        self.filename = filename
        
    def setup(self, worker):
        self.worker = worker
        import boto3
        import click
        from s3urls import parse_url
        import tarfile

        import logging, traceback, sys, os    
        logger = logging.getLogger("embeddings microservice")
        logger.debug("downloading file...")
        vocab_parsed_url = parse_url(self.s3Url)
        s3 = boto3.client('s3')
        s3.download_file(vocab_parsed_url['bucket'], vocab_parsed_url['key'], self.filename)
        logger.debug("done downloading....")
        logger.debug("extracting....")
        if self.filename.endswith("tar.gz") or self.filename.endswith("tgz"):
            tar = tarfile.open(self.filename, "r:gz")
            tar.extractall(path='/tmp/')
            tar.close()
            os.remove(self.filename)
            logger.debug("done extracting....")

In [14]:
!pip install boto3

     |████████████████████████████████| 7.7 MB 4.5 MB/s eta 0:00:01


In [20]:
!pip install s3urls

In [21]:
tgz_file_global = "node_repo.tgz"
repo_folder_on_worker = "/tmp/repo"
embeddings_inputs_data_folder = "node_emb_inputs"

In [22]:
s3Url=f"s3://d2v-tmp/{tgz_file_global}"
filename=tgz_file_global

In [23]:
import boto3
import click
from s3urls import parse_url
import tarfile

import logging, traceback, sys, os    
vocab_parsed_url = parse_url(s3Url)
s3 = boto3.client('s3')
s3.download_file(vocab_parsed_url['bucket'], vocab_parsed_url['key'], filename)

In [4]:
from hyperplane import notebook_common as nc
client, cluster = nc.initialize_cluster(num_workers=2)

👉 Hyperplane: selecting worker node pool
👉 Hyperplane: selecting scheduler node pool
Creating scheduler pod on cluster. This may take some time.
👉 Hyperplane: spinning up a dask cluster with a scheduler as a standalone container.
👉 Hyperplane: In a few minutes you'll be able to access the dashboard at https://ds.hyperplane.dev/dask-cluster-58729332-3dae-436b-814c-8b1f51266943/status
👉 Hyperplane: to get logs from all workers, do `cluster.get_logs()`


In [7]:
from dask.distributed import PipInstall

In [24]:
plugin = PipInstall(packages=["boto3", "s3urls", "botocore", "click", "tarfile"], pip_options=["--upgrade"])
client.register_worker_plugin(plugin)

{'tcp://10.1.193.3:32895': {'status': 'repeat'},
 'tcp://10.1.193.3:34039': {'status': 'repeat'},
 'tcp://10.1.193.3:34405': {'status': 'repeat'},
 'tcp://10.1.193.3:35445': {'status': 'repeat'},
 'tcp://10.1.193.3:38539': {'status': 'repeat'},
 'tcp://10.1.193.3:39927': {'status': 'repeat'},
 'tcp://10.1.193.3:40353': {'status': 'repeat'},
 'tcp://10.1.193.3:41199': {'status': 'repeat'},
 'tcp://10.1.193.3:41675': {'status': 'repeat'},
 'tcp://10.1.193.3:41803': {'status': 'repeat'},
 'tcp://10.1.193.3:43879': {'status': 'repeat'},
 'tcp://10.1.193.3:44115': {'status': 'repeat'},
 'tcp://10.1.193.3:44501': {'status': 'repeat'},
 'tcp://10.1.193.3:44903': {'status': 'repeat'},
 'tcp://10.1.193.3:45615': {'status': 'repeat'},
 'tcp://10.1.194.3:33727': {'status': 'repeat'},
 'tcp://10.1.194.3:34297': {'status': 'repeat'},
 'tcp://10.1.194.3:35349': {'status': 'repeat'},
 'tcp://10.1.194.3:36365': {'status': 'repeat'},
 'tcp://10.1.194.3:36875': {'status': 'repeat'},
 'tcp://10.1.194.3:4

In [25]:
client.register_worker_plugin(S3DownloadPlugin(s3Url=f"s3://d2v-tmp/{tgz_file_global}", filename=tgz_file_global))

NoCredentialsError: Unable to locate credentials